<img src='https://github.com/Ikomia-dev/notebooks/blob/main/examples/img/banner_ikomia.png?raw=true'>




# Easy semantic segmentation with MMSegmentation

**MMSegmentation** is part of the OpenMMLab project and is developed by the Multimedia Laboratory at the Chinese University of Hong Kong. It specializes in semantic segmentation, a vital component in the field of computer vision. 

It offers an extensive collection of segmentation models and algorithms, making it a go-to choice for both researchers and practitioners in the field.



![MMSegmentation illustration](https://github.com/open-mmlab/mmsegmentation/blob/main/resources/seg_demo.gif?raw=true)

## Setup

You need to install Ikomia Python API with pip


In [ ]:
!pip install ikomia

## Run MMSegmentation on your image

---

**-Google Colab ONLY- Restart runtime after the first run of the workflow below** 

Click on the "RESTART RUNTIME" button at the end the previous window.

---

In [ ]:
from ikomia.dataprocess.workflow import Workflow

# Init your workflow
wf = Workflow()

# Add object detection algorithm
segmentor = wf.add_task(name="infer_mmlab_segmentation", auto_connect=True)


segmentor.set_parameters({
        "model_name": "pspnet",
        "model_config": "pspnet_r50-d8_4xb2-40k_cityscapes-512x1024.py",
        "cuda": "True",
    })


# Run the workflow on image
wf.run_on(url="https://github.com/open-mmlab/mmsegmentation/blob/main/demo/demo.png?raw=true")


In [ ]:
from ikomia.core import IODataType
from ikomia.utils.displayIO import display

from PIL import ImageShow
ImageShow.register(ImageShow.IPythonViewer(), 0)

# Display the results
display(segmentor.get_image_with_mask())

## List of parameters

- **model_name** (str, default="maskformer"): model name. 
- **model_config** (str, default="maskformer_r50-d32_8xb2-160k_ade20k-512x512"): name of the model configuration file.
- **config_file** (str, default=""): path to model config file (only if *use_custom_model=True*). The file is generated at the end of a custom training. Use algorithm ***train_mmlab_detection*** from Ikomia HUB to train custom model.
- **model_weight_file** (str, default=""): path to model weights file (.pt) (only if *use_custom_model=True*). The file is generated at the end of a custom training.
- **cuda** (bool, default=True): CUDA acceleration if True, run on CPU otherwise.

MMLab framework for object detection and instance segmentation offers a large range of models. To ease the choice of couple (model_name/model_config), you can call the function *get_model_zoo()* to get a list of possible values.


In [ ]:
from ikomia.dataprocess.workflow import Workflow

# Init your workflow
wf = Workflow()

# Add object detection algorithm
segmentor = wf.add_task(name="infer_mmlab_segmentation", auto_connect=True)

# Get list of possible models (model_name, model_config)
print(segmentor.get_model_zoo())